In [14]:
%load_ext autoreload
%autoreload 2 
# !apt-get install -y xvfb

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
import pandas as pd
import numpy as np


In [16]:
df = pd.read_excel("airway_tree_pilot.xlsx", sheet_name=0)

In [17]:
df.drop(columns = ["manufacturer", "model"], inplace=True)

In [18]:
df.describe()


,idno,startbpid,endbpid,centerlinelength,angle,dircosx,dircosy,dircosz,avginnerarea,generation,weibel_generation
count,7.768300e+04,77683.000000,77683.000000,77683.000000,77454.000000,77683.000000,77683.000000,77683.000000,77683.000000,77683.000000,77683.000000
mean,5.986778e+06,149.658226,197.128509,10.614453,143.201617,-0.021328,0.056368,0.138202,11.161638,6.539384,7.338839
std,1.832951e+06,121.756964,130.566550,9.629416,24.407202,0.552013,0.542999,0.614584,25.017431,1.918851,2.273523
min,3.011950e+06,-1.000000,1.000000,0.498047,4.410540,-1.000000,-1.000000,-1.000000,-1.000000,0.000000,0.000000
25%,4.017080e+06,51.000000,92.000000,4.138575,132.688500,-0.476731,-0.381630,-0.392263,4.149160,5.000000,6.000000
50%,7.012039e+06,120.000000,180.000000,8.742300,149.928000,0.000000,0.061056,0.229416,6.168230,7.000000,7.000000
75%,8.010528e+06,220.000000,280.000000,14.169850,160.462500,0.436436,0.497468,0.704271,10.071250,8.000000,9.000000
max,8.023336e+06,744.000000,747.000000,128.734000,180.000000,1.000000,1.000000,1.000000,2343.240000,16.000000,17.000000


In [39]:
print("There are 227 participants, each participant has on average 342 \pm 110 branch segments")

df.groupby("idno").agg(
    {
        "anatomicalname":"count"
    }
).rename(
    columns = {
        "anatomicalname":"num_branch_segments"
    }
).sort_values(
    by="num_branch_segments",
    ascending=False
).describe().round(0).astype(int)



There are 227 participants, each participant has on average 342 \pm 110 branch segments


,num_branch_segments
count,227
mean,342
std,110
min,98
25%,264
50%,331
75%,408
max,744


In [20]:
print("Number of weibel generations per participant")

weibel = df.groupby("idno").agg(
    {
        "weibel_generation":["max", "mean"]
    }
)






new_cols = [first+"_"+second for (first, second) in weibel.columns]
weibel.columns = new_cols
weibel.describe().round().astype(int)

Number of weibel generations per participant


,weibel_generation_max,weibel_generation_mean
count,227,227
mean,13,7
std,1,1
min,9,5
25%,12,7
50%,13,7
75%,14,8
max,17,9


In [32]:
print(f"{(df['anatomicalname']=='-').sum()/len(df)*100:.2f}% of branch segments dont have anatomical names")

df.groupby("anatomicalname").agg({"idno":"count"}).sort_values(by="idno", ascending=False)

# what is unnamed?

89.77% of branch segments dont have anatomical names


,idno
anatomicalname,
-,69737
LUL,227
Trachea,227
RUL,227
RMB,227
...,...
unnamed-169,1
unnamed-4,1
unnamed-395,1


In [38]:
print("All branches start at trachea and end on segment number 350\pm 113")
df.groupby("idno").agg({"startbpid":"min", "endbpid":max}).describe().round()

All branches start at trachea and end on segment number 350\pm 113


,startbpid,endbpid
count,227.0,227.0
mean,-1.0,350.0
std,0.0,113.0
min,-1.0,101.0
25%,-1.0,270.0
50%,-1.0,335.0
75%,-1.0,414.0
max,-1.0,747.0
